In [1]:
import pandas as pd
import numpy as np

from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so

import warnings
warnings.filterwarnings('ignore')

## 3-1 표본추출
### 단순랜덤추출법
- 각 샘플에 번호를 부여하여 임의의 n개를 추출, 샘플이 추출 될 확률은 동일

In [ ]:
data_list = [1,2,3,4,5, 'a', 'b', 'c']
print("random.sample: ", random.sample(data_list, 4))
print("np.random.choice", np.random.choice(data_list, 4, replace=True))
print("0~10 사이 정수 중 난수 3개 생성: ", np.random.randint(0,10,3))
print("0~1 사이 실수 2*2 배열로 생성: ", np.random.rand(2,2))

### 계통추출법
- 번호를 부여한 샘플은 나열하고, 구간을 N/n으로 나눈다. 각 구간의 임의로 샘플을 선택하고 특정 인터벌 단위로 샘플 추출

In [ ]:
ata, n = iris, 8; N = len(data); K = N//n
index = data[:K].sample(1).index

sys_df = pd.DataFrame()
while len(sys_df) < n:
    sys_df = sys_df.append(data.loc[index, :])
    index += K

### 집락추출법
- 군집별로 랜덤추출법 수행
    - 지역표본추출, 다단계표본추출 : 군집 내 요소들은 상이하지만 군집과 군집은 비교적 유사
    - 층화추출법에서 층을 집락으로 대치
    
### 층화추출법
- 계층별로 랜덤추출법 수행 : 각 층 내 요소들은 유사하지만, 층과 층의 요소들은 상이
    - 비례층화추출법, 불비례층화추출법

In [ ]:
# 비례층화추출법 : 원본 데이터의 비율대로 추출
data, stratum, sampling_n = iris, 'target', 9

levels = data[stratum].unique()
total = data[stratum].value_counts().sum()
prop_val = data[stratum].value_counts()/total
n = prop_val * sampling_n

result = pd.DataFrame()
for level in levels:
    temp_df = data[data[stratum]==level].sample(int(n[level]))
    result = pd.concat([result, temp_df])
print(result)

In [ ]:
# 불비례층화추출법 : 임의로 정한 특정비율대로 샘플링
data, stratum, sampling_n, proportion = iris, 'target', 10, {0:0.2, 1:0.5, 2:0.3}

levels = list(proportion.keys())
prop_val = np.array(list(proportion.values()))
total = sum(prop_val)
n = prop_val * sampling_n

result = pd.DataFrame()
for level in levels:
    temp_df = data[data[stratum]==level].sample(int(n[level]))
    result = pd.concat([result, temp_df])
print(result)

## 3-2 데이터분할
- 과적합을 피함으로써 일반화된 성능확보
### 일반적 데이터 분할 및 홀드아웃

In [3]:
from sklearn.datasets import load_iris

data = load_iris()
iris_cols = list(data['feature_names']) + ['target']
iris = pd.DataFrame(np.c_[data['data'], data['target']], columns=[col.replace(" (cm)", '') for col in iris_cols])


In [4]:
from sklearn.model_selection import train_test_split

X = iris.drop('target', axis=1)
y = iris['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Shuffle Split
- 무작위 순열 교차 검증
    - 데이터 크기가 작은 경우, 분할 샘플들이 유사할 수 있다.

In [6]:
from sklearn.model_selection import ShuffleSplit
from collections import Counter

ss = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=4)
for i, (train_idx, test_idx) in enumerate(ss.split(X)):
    X_train, X_test, y_train, y_test = X.iloc[train_idx, :], X.iloc[test_idx, :], y.iloc[train_idx], y.iloc[test_idx]

### K-fold 분할
- 데이터를 k개 집단으로 나눈 뒤, k-1개 집단을 학습용, 1개 집단을 검증용으로 사용

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=False)
for i, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train, X_test, y_train, y_test = X.iloc[train_idx, :], X.iloc[test_idx, :], y.iloc[train_idx], y.iloc[test_idx]

### Stratified K-Fold
- KFold 과정 중에서, 집단 간 클래스 불균형을 막기 위해, 타겟 변수 클래스(범주형 타겟) 들이 일정한 비율로 배치되도록 함

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=4)
for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train, X_test, y_train, y_test = X.iloc[train_idx, :], X.iloc[test_idx, :], y.iloc[train_idx], y.iloc[test_idx]

### Group K-Fold 분할
- 범주형 변수인 group 수준별 데이터를 각 분할마다 검증용 데이터로 진행하도록 하는 방법
- group 수준의 개수는 fold 갯수 보다 같거나 커야 함

In [ ]:
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=4)
# 분할 시, group 고려하므로 split의 group를 입력
for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, group)):
    X_train, X_test, y_train, y_test = X.iloc[train_idx, :], X.iloc[test_idx, :], y.iloc[train_idx], y.iloc[test_idx]

## 3-3 교차검증
- 학습용 데이터와 검증용 데이터를 앞서 언급한 분할 방법들로 다양하게 분할하고, 여러 파라미터 조건 하에 학습하여 교차검증함으로써 최적화된 일반화 모델 완성

### 분할 샘플들로 교차검증

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression

X = iris.drop('target', axis=1)
y = iris['target']

logreg = LogisticRegression(max_iter=300, C=0.1)
skf = StratifiedKFold(n_splits=4)
result = cross_validate(logreg, X, y, cv=skf, return_train_score=True)
print(pd.DataFrame(result))

### 파라미터 후보들로 교차검증

In [ ]:
from sklearn.model_selection import GridSearchCV

logreg = LogisticRegression(max_iter=300)
param_grid = {'C':[0.01, 0.1, 1], 'solver':['lbfgs', 'liblinear']}
skf = StratifiedKFold(n_splits=4)
grid = GridSearchCV(logreg, param_grid, cv=skf)
grid.fit(X,y)

print(pd.DataFrame(grid.cv_results_))